## 실시간 엔드포인트를 DJL Serving을 이용해 호스팅
이번 노트북에서는 [Deep Java Library](https://djl.ai/) (DJServing)가 지원하는 대형 모델 추론 컨테이너(LMI)를 모델 서빙 솔루션으로 사용합니다. 

AWS Neuron SDK는 사용자가 Inferentia 장치의 강력한 처리 능력을 쉽게 활용할 수 있게 해주며, DJLServing은 Java 기반 환경에서 대규모 모델을 손쉽게 서빙할 수 있도록 지원합니다. 이 노트북은 Amazon Elastic Compute Cloud(Amazon EC2) inf2.xlarge 인스턴스에 TinyLLama 모델을 배포합니다.

## 컨테이너 이미지 확인

LMI 컨테이너는 [DJLServing](https://github.com/deepjavalibrary/djl-serving)을 사용합니다. DJLServing은 [transformers-neuronx](https://github.com/aws-neuron/transformers-neuronx) 라이브러리와 통합되어 있어, NeuronCores 간의 텐서 병렬 처리를 지원하는 모델 서버입니다. DJL 모델 서버와 transformers-neuronx 라이브러리는 컨테이너의 핵심 구성 요소로 작용하며, [Neuron SDK](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/)도 포함되어 있습니다. 이 설정은 모델을 [AWS Inferentia2](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-hardware/inferentia2.html) 가속기에 로드하고, 여러 [NeuronCores](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-hardware/neuron-core-v2.html#neuroncores-v2-arch)에 걸쳐 모델을 병렬 처리하며, HTTP 엔드포인트를 통해 서비스를 제공하는 데 도움이 됩니다. 이는 SageMaker 라이브러리를 사용합니다.

In [2]:
# NOTE: 다음 코드를 통해 이미지 주소를 가져올 수 있지만, 여기선 건너뜁니다.

# import sagemaker
# from sagemaker import Model, image_uris, serializers
# import boto3

# boto3_sess = boto3.Session(region_name="ap-northeast-1")

# sess = sagemaker.session.Session(boto_session = boto3_sess)  # sagemaker session for interacting with different AWS APIs
# role = sagemaker.get_execution_role()  # execution role for the endpoint

# image_uri = image_uris.retrieve(
#         framework="djl-neuronx",
#         region=sess.boto_session.region_name,
#         version="0.24.0"
#     )

image_uri = "763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/djl-inference:0.24.0-neuronx-sdk2.14.1"
image_uri

'763104351884.dkr.ecr.ap-northeast-1.amazonaws.com/djl-inference:0.24.0-neuronx-sdk2.14.1'

## 모델 서빙 아티팩트 준비하기

LMI 컨테이너는 Amazon Simple Storage Service (Amazon S3) 버킷 또는 Hugging Face Hub에서 모델을 로드하는 것을 지원합니다. 이때는 S3 또는 HuggingFace의 SafeTensor 형식의 모델이, 컨테이너 내에서 시작 전 자동으로 컴파일 됩니다. 모델을 로드하고 호스팅하기 위해 *`serving.properties`* 파일에 필요한 파라미터들이 필요합니다. 모델 로딩 시 사용될 수 있도록 serving.properties 파일에 구성 가능한 파라미터를 제공할 수 있습니다. 구성 가능한 파라미터의 전체 목록은 [All DJL configuration options](https://github.com/deepjavalibrary/djl-serving/blob/master/serving/docs/configurations.md)에서 참조하세요. (참고: 직접 model.py 파일을 작성하고 서빙 코드에 사용하도록 할 수도 있지만, 이 경우 DJLServing API와 transformers-neuronx API 간의 연결 다리 역할을 할 모델 로딩 및 추론 메소드를 구현해야 합니다.)

In [22]:
# 모델 서버에 필요한 serving.properties 파일 생성 (Download from HuggingFace or S3)

file_content = f"""engine=Python
option.entryPoint=djl_python.transformers_neuronx
option.model_id=TinyLlama/TinyLlama-1.1B-Chat-v1.0
option.batch_size=1
option.neuron_optimize_level=1
option.tensor_parallel_degree=2
option.load_in_8bit=false
option.n_positions=512
option.rolling_batch=auto
option.dtype=fp16"""

with open("serving.properties","w") as f:
    f.write(file_content)


In [ ]:
%%sh

# `serving.properties`를 models/tinyllama 디렉토리로 이동
rm -rf models 
rm -rf logs
mkdir -p models/tinyllama logs
cp serving.properties models/tinyllama/

## 컨테이너 생성
다음으로, 앞서 정의한 모델 설정을 사용하여 컨테이너 엔드포인트를 생성합니다. 모델 배포는 보통 4-5분 정도 소요되며, 이 과정에서 모델이 컴파일됩니다. (HuggingFace 또는 S3를 model_id로 지정한 경우)

In [3]:
!aws ecr get-login-password --region ap-northeast-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.ap-northeast-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [ ]:
!docker run -it --rm  --rm --network=host \
  -v `pwd`/models:/opt/ml/model/ \
  -v `pwd`/logs:/opt/djl/logs \
  -v `pwd`/TinyLlama-neuron:/models/TinyLlama-neuron \
  -u djl \
  --device /dev/neuron0 \
  --device /dev/neuron1 \
  --device /dev/neuron2 \
  --device /dev/neuron3 \
  --device /dev/neuron4 \
  --device /dev/neuron5 \
  --device /dev/neuron6 \
  --device /dev/neuron7 \
  --device /dev/neuron8 \
  --device /dev/neuron9 \
  --device /dev/neuron10 \
  --device /dev/neuron11 \
  -e MODEL_LOADING_TIMEOUT=7200 \
  -e PREDICT_TIMEOUT=360 \
  {image_uri} serve

INFO  ModelServer Starting model server ...
INFO  Ec2Utils DJL will collect telemetry to help us better understand our users? needs, diagnose issues, and deliver additional features. If you would like to learn more or opt-out please go to: https://docs.djl.ai/docs/telemetry.html for more information.
INFO  ModelServer Starting djl-serving: 0.24.0 ...
INFO  ModelServer 
Model server home: /opt/djl
Current directory: /opt/djl
Temp directory: /tmp
Command line: -Dlog4j.configurationFile=/usr/local/djl-serving-0.24.0/conf/log4j2.xml -Xmx1g -Xms1g -Xss2m -XX:+ExitOnOutOfMemoryError
Number of CPUs: 192
Number of Neuron cores: 24
Max heap size: 1024
Config file: /opt/djl/conf/config.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Default job_queue_size: 1000
Default batch_size: 1
Default max_batch_delay: 100
Default max_idle_time: 60
Model Store: /opt/ml/model
Initial Models: ALL
Netty threads: 0
Maximum Request Size: 67108864
Environment variables:
	

## 추론 테스트
Docker 엔드포인트가 생성된 후, Predictor 객체를 사용하여 Docker 엔드포인트에 대해 실시간 예측을 수행할 수 있습니다.
- 추론 요청을 제출하고 응답을 받기 위해 `curl` 명령어를 사용합니다.
- 요청과 응답은 JSON 형식으로 이루어집니다.

In [ ]:
curl -X POST "http://127.0.0.1:8080/predictions/tinyllama" \
     -H 'Content-Type: application/json' \
     -d '{
         "seq_length": 512,
         "inputs": "Welcome to Amazon Elastic Compute Cloud",
         "parameters": {
             "max_new_tokens": 32,
             "do_sample": "true"
         }
     }'


터미널을 열고 모델 서버에 추론 요청을 제출하고 추론 결과를 받아봅시다.